In [1]:
import psycopg2
from datetime import datetime
import pandas as pd
import numpy as np

conn = psycopg2.connect(dbname='test_data', user='pauloarantes', host='/tmp')
c = conn.cursor()

In [26]:
c.execute(
    '''
SELECT
u.id,
u.last_sign_in_at::date,
u.created_at::date,
to_timestamp(CAST(bs.max_time as int))::date as last_session,
bs.num_sessions,
bs.total_artists_followed,
bs.total_artworks_favorited,
bs.total_artworks_shared,
bs.total_follows,
bs.total_favorites,
bs.last_favorited_artwork_date,
bs.last_followed_artist_date,
city.city,
gender.gender,
user_type.user_type,
os.os

FROM users u

JOIN (SELECT CAST(data->'properties'->>'user id' AS int) as user_id,
             COUNT(data->'properties'->>'user id') as num_sessions,
             MAX(data->'properties'->>'time') as max_time,
             MAX(data->'properties'->>'total artists followed') as total_artists_followed,
             MAX(data->'properties'->>'total artworks favorited')as total_artworks_favorited,
             MAX(data->'properties'->>'total artworks shared')as total_artworks_shared,
             MAX(data->'properties'->>'last favorited artwork date') as last_favorited_artwork_date,
             MAX(data->'properties'->>'last followed artist date') as last_followed_artist_date,
             MAX(data->'properties'->>'total follows') as total_follows,
             MAX(data->'properties'->>'total favorites') as total_favorites
        FROM mp_began_session
        GROUP BY data->'properties'->>'user id') bs
ON bs.user_id=u.id

FULL OUTER JOIN user_city city
ON u.id=city.user_id

FULL OUTER JOIN user_gender gender
ON u.id=gender.user_id

FULL OUTER JOIN user_os os
ON u.id=os.user_id

FULL OUTER JOIN user_type
ON u.id=user_type.user_id

'''
)

In [27]:
data = c.fetchall()
print len(data)

57931


In [28]:
cols = ['id',
'last_sign_in_at',
'created_at',
'last_session',
'num_sessions',
'total_artists_followed',
'total_artworks_favorited',
'total_artworks_shared',
'total_follows',
'total_favorites',
'last_favorited_artwork_date',
'last_followed_artist_date',
'city',
'gender',
'user_type',
'os']

In [29]:
df = pd.DataFrame(data, columns=cols)

In [45]:
df.created_at = pd.to_datetime(df.created_at).dt.date
df.last_favorited_artwork_date = pd.to_datetime(df.last_favorited_artwork_date, errors='coerce').dt.date
df.last_followed_artist_date = pd.to_datetime(df.last_followed_artist_date, errors='coerce').dt.date

In [46]:
df.head()

,id,last_sign_in_at,created_at,last_session,num_sessions,total_artists_followed,total_artworks_favorited,total_artworks_shared,total_follows,total_favorites,last_favorited_artwork_date,last_followed_artist_date,city,gender,user_type,os
0,10002.0,2014-05-11,2014-05-11,2015-08-01,1.0,0,2,0,0,2,NaN,NaN,Arlington Heights,None,None,iPhone OS
1,10006.0,None,2014-05-11,2016-02-28,9.0,0,3,0,0,3,NaN,NaN,Elizabeth,male,None,iPhone OS
2,10013.0,None,2014-05-11,2015-06-06,2.0,0,1,0,None,None,NaN,NaN,Philadelphia,None,Logged in with FB,iPhone OS
3,10045.0,2014-05-11,2014-05-11,2016-03-16,49.0,0,13,0,None,None,NaN,NaN,Jersey City,None,None,iPhone OS
4,10071.0,2014-05-11,2014-05-11,2016-04-17,2.0,0,2,0,None,None,NaN,NaN,San Antonio,None,Organic,iPhone OS


In [52]:
df.to_csv('dataset.csv')

In [ ]:
c.execute(
'''
CREATE TABLE cities AS
SELECT data->'properties'->>'user id' as user_id, 
       MAX(data->'properties'->>'time') as max_time
FROM mp_began_session 
WHERE data->'properties'->>'$city' IS NOT NULL 
GROUP BY data->'properties'->>'user id'
;
'''
)

In [ ]:
c.execute('''
WITH genders AS(
        SELECT data->'properties'->>'user id' as user_id, 
               MAX(data->'properties'->>'time') as max_time
        FROM mp_began_session 
        WHERE data->'properties'->>'gender' IS NOT NULL 
        GROUP BY data->'properties'->>'user id'
        )
                SELECT  CAST(mp.data->'properties'->>'user id' AS int) as user_id,
                             mp.data->'properties'->>'gender' as gender
                FROM mp_began_session mp
                JOIN genders
                ON genders.user_id=mp.data->'properties'->>'user id' 
                AND genders.max_time=mp.data->'properties'->>'time'
''')

In [ ]:
SELECT
u.id,
u.last_sign_in_at::date,
u.created_at::date,
to_timestamp(CAST(bs.max_time as int)) as last_session,
bs.num_sessions,
bs.total_artists_followed,
bs.total_artworks_favorited,
bs.total_artworks_shared,
bs.total_follows,
bs.total_favorites,
bs.last_favorited_artwork_date::date,
bs.last_followed_artist_date::date,
city.city 

INTO test_city

FROM users u

JOIN (SELECT CAST(data->'properties'->>'user id' AS int) as user_id,
             COUNT(data->'properties'->>'user id') as num_sessions,
             MAX(data->'properties'->>'time') as max_time,
             MAX(data->'properties'->>'total artists followed') as total_artists_followed,
             MAX(data->'properties'->>'total artworks favorited')as total_artworks_favorited,
             MAX(data->'properties'->>'total artworks shared')as total_artworks_shared,
             MAX(data->'properties'->>'last favorited artwork date') as last_favorited_artwork_date,
             MAX(data->'properties'->>'last followed artist date') as last_followed_artist_date,
             MAX(data->'properties'->>'total follows') as total_follows,
             MAX(data->'properties'->>'total favorites') as total_favorites
        FROM mp_began_session
        GROUP BY data->'properties'->>'user id') bs
ON bs.user_id=u.id

JOIN (SELECT  CAST(mp.data->'properties'->>'user id' AS int) as user_id,
                   mp.data->'properties'->>'$city' as city
                FROM mp_began_session mp
                JOIN user_cities_time
                ON user_cities_time.user_id=mp.data->'properties'->>'user id' 
                AND user_cities_time.max_time=mp.data->'properties'->>'time') city
ON city.user_id=u.id;


In [ ]:
c.execute(
    '''
SELECT
u.id,
u.last_sign_in_at::date,
u.created_at::date,
to_timestamp(CAST(bs.max_time as int)) as last_session,
bs.num_sessions,
bs.total_artists_followed,
bs.total_artworks_favorited,
bs.total_artworks_shared,
bs.total_follows,
bs.total_favorites,
bs.last_favorited_artwork_date::date,
bs.last_followed_artist_date::date,
city.city 

FROM users u

JOIN (SELECT CAST(data->'properties'->>'user id' AS int) as user_id,
             COUNT(data->'properties'->>'user id') as num_sessions,
             MAX(data->'properties'->>'time') as max_time,
             MAX(data->'properties'->>'total artists followed') as total_artists_followed,
             MAX(data->'properties'->>'total artworks favorited')as total_artworks_favorited,
             MAX(data->'properties'->>'total artworks shared')as total_artworks_shared,
             MAX(data->'properties'->>'last favorited artwork date') as last_favorited_artwork_date,
             MAX(data->'properties'->>'last followed artist date') as last_followed_artist_date,
             MAX(data->'properties'->>'total follows') as total_follows,
             MAX(data->'properties'->>'total favorites') as total_favorites
        FROM mp_began_session
        GROUP BY data->'properties'->>'user id') bs
ON bs.user_id=u.id

JOIN (SELECT  CAST(mp.data->'properties'->>'user id' AS int) as user_id,
                   mp.data->'properties'->>'$city' as city
                FROM mp_began_session mp
                JOIN user_cities_time
                ON user_cities_time.user_id=mp.data->'properties'->>'user id' 
                AND user_cities_time.max_time=mp.data->'properties'->>'time') city
ON city.user_id=u.id;
'''
)

In [ ]:
data = c.fetchall()
print len(data)

In [ ]:
type(data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
WITH cities AS(
        SELECT data->'properties'->>'user id' as user_id, 
               MAX(data->'properties'->>'time') as max_time
        FROM mp_began_session 
        WHERE data->'properties'->>'$city' IS NOT NULL 
        GROUP BY data->'properties'->>'user id'
        )
                SELECT  CAST(mp.data->'properties'->>'user id' AS int) as user_id,
                             mp.data->'properties'->>'$city' as city
                INTO user_city
                FROM mp_began_session mp
                JOIN user_cities_time c
                ON c.user_id=mp.data->'properties'->>'user id' 
                AND c.max_time=mp.data->'properties'->>'time'